In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from pathlib import Path
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
def loadmodel():
    model = CatBoostRegressor()
    model.load_model("catboost_best.cbm")
    params = model.get_params()
    return model, params


In [32]:
def get_dfs(df):
    X_full = df.drop(['exam_score', 'id'], axis=1)
    y_full = df['exam_score']
    cat_vars = X_full.select_dtypes(include="object").columns.tolist()
    return X_full, y_full, cat_vars

In [18]:
def start_model(X, y, cat_features):
    final_model = CatBoostRegressor(**params)
    final_model.fit(X, y, verbose=100, cat_features=cat_vars)
    return final_model

In [35]:
def feature_changer(df):
    df['study_sleeping_range'] = df['study_hours'] * df['sleep_hours']
    


In [35]:
df = pd.read_csv("train.csv")

X_full = df.drop("exam_score", axis=1)
y_full = df['exam_score']
cat_vars = X_full.select_dtypes(include="object").columns.tolist()

In [39]:
final_model = CatBoostRegressor(**params)

In [40]:
final_model.fit(X_full, y_full, verbose=100, cat_features=cat_vars)

0:	learn: 18.4267617	total: 477ms	remaining: 27m 50s
100:	learn: 8.9313880	total: 30s	remaining: 16m 49s
200:	learn: 8.8428118	total: 56.2s	remaining: 15m 22s
300:	learn: 8.8314714	total: 1m 20s	remaining: 14m 14s
400:	learn: 8.8210595	total: 1m 45s	remaining: 13m 31s
500:	learn: 8.8135394	total: 2m 8s	remaining: 12m 47s
600:	learn: 8.8068587	total: 2m 32s	remaining: 12m 15s
700:	learn: 8.8008715	total: 2m 56s	remaining: 11m 44s
800:	learn: 8.7952945	total: 3m 21s	remaining: 11m 19s
900:	learn: 8.7907941	total: 3m 46s	remaining: 10m 54s
1000:	learn: 8.7857717	total: 4m 12s	remaining: 10m 29s
1100:	learn: 8.7816088	total: 4m 37s	remaining: 10m 3s
1200:	learn: 8.7772390	total: 5m 2s	remaining: 9m 39s
1300:	learn: 8.7732455	total: 5m 27s	remaining: 9m 14s
1400:	learn: 8.7695046	total: 5m 53s	remaining: 8m 49s
1500:	learn: 8.7654468	total: 6m 20s	remaining: 8m 26s
1600:	learn: 8.7613687	total: 6m 46s	remaining: 8m 2s
1700:	learn: 8.7578831	total: 7m 13s	remaining: 7m 38s
1800:	learn: 8.754

In [59]:
X_test = pd.read_csv("test.csv")
X_test

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty
0,630000,24,other,ba,6.85,65.2,yes,5.2,poor,group study,high,easy
1,630001,18,male,diploma,6.61,45.0,no,9.3,poor,coaching,low,easy
2,630002,24,female,b.tech,6.60,98.5,yes,6.2,good,group study,medium,moderate
3,630003,24,male,diploma,3.03,66.3,yes,5.7,average,mixed,medium,moderate
4,630004,20,female,b.tech,2.03,42.4,yes,9.2,average,coaching,low,moderate
...,...,...,...,...,...,...,...,...,...,...,...,...
269995,899995,21,other,b.com,2.55,82.3,yes,8.4,average,mixed,medium,hard
269996,899996,17,female,b.com,0.49,46.4,yes,8.8,good,mixed,low,easy
269997,899997,22,male,bba,6.62,74.7,yes,5.5,good,coaching,high,easy
269998,899998,22,other,ba,4.08,51.8,yes,8.7,poor,online videos,high,moderate


In [60]:
predict = final_model.predict(X_test)

In [61]:
id = X_test['id']

In [57]:
predict_df = pd.DataFrame({
    "id": id,
    "exam_score": predict
})

Hello


In [62]:
print(predict_df)

            id  exam_score
0       630000   72.528213
1       630001   70.313329
2       630002   86.652140
3       630003   57.241950
4       630004   46.161821
...        ...         ...
269995  899995   60.652624
269996  899996   38.582182
269997  899997   91.400383
269998  899998   55.319106
269999  899999   67.898307

[270000 rows x 2 columns]


In [68]:
predict_df.to_csv("pred_from_exam_score2.csv", index=False)


In [136]:
df = pd.read_csv('train.csv')

In [137]:
df = feature_changer(df)

In [138]:
df.head()

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score,study_sleeping_range,sleep_quality_ord,facility_ord,attendance_bins
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3,4.844875,2,1,5
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7,2.908125,1,2,5
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.0,3.393000,1,3,5
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.9,2.075000,2,3,3
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.0,9.180000,3,3,5


In [139]:
model, params = loadmodel()

In [140]:
params['iterations'] = 1100

In [141]:
X_full, y_full, cat_vars = get_dfs(df)

In [142]:
model = start_model(X_full, y_full, cat_vars)

0:	learn: 18.4130546	total: 484ms	remaining: 8m 51s
100:	learn: 8.9283779	total: 28.5s	remaining: 4m 41s
200:	learn: 8.8518513	total: 55.8s	remaining: 4m 9s
300:	learn: 8.8353348	total: 1m 22s	remaining: 3m 39s
400:	learn: 8.8210505	total: 1m 50s	remaining: 3m 12s
500:	learn: 8.8130527	total: 2m 16s	remaining: 2m 42s
600:	learn: 8.8061705	total: 2m 41s	remaining: 2m 14s
700:	learn: 8.8005354	total: 3m 8s	remaining: 1m 47s
800:	learn: 8.7943083	total: 3m 34s	remaining: 1m 20s
900:	learn: 8.7889624	total: 4m 1s	remaining: 53.3s
1000:	learn: 8.7836505	total: 4m 27s	remaining: 26.5s
1099:	learn: 8.7788353	total: 4m 54s	remaining: 0us


In [143]:
df.head(1)

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score,study_sleeping_range,sleep_quality_ord,facility_ord,attendance_bins
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3,4.844875,2,1,5


In [144]:

# Modelo com 1100 iterations, study_hours * sleep_hours, RMSE = 8.7821870
# Modelo com 1100 iterations, study_hours e cross facility exam difficulty, RMSE = 8.7823824
# Modelo com 1100 iterations, study_hours com sqrt, RMSE = 8.7835250
# Modelo com 1100 iterations, study_hours com sleep_hours/8, RMSE = 8.7821870
# Modelo com 1100 iterations, study_hours com sleep quality order, RMSE = 8.7793982
# Modelo com 1100 iterations, study_hours com sleep quality order e facility order RMSE = 8.7785620
# Modelo com 1100 iterations, study_hours com sleep quality order, facility order e exam_difficulty_ord RMSE = 8.777288
# Modelo com 1100 iterations, study_hours com sleep quality order, facility order e exam_difficulty_ord, bin em class_attendance com int RMSE = 8.7788353





In [135]:
def feature_changer(df):
    df['study_sleeping_range'] = (df['study_hours'] * df['sleep_hours']/8)
    df['sleep_quality_ord'] = df['sleep_quality'].map({
        'poor': 1,
        'average': 2,
        'good': 3})
    
    df['facility_ord'] = df['facility_rating'].map({
        'low': 1,
        'medium': 2,
        'high': 3
        })
           
    df['attendance_bins'] = pd.cut(
        df['class_attendance'],
        bins=[0, 20, 40, 60, 80, 100],
        labels=[1,2,3,4,5]
        ).astype(int)
        
    return df

In [107]:
X_full['exam_difficulty'].value_counts()

exam_difficulty
moderate    353982
easy        176540
hard         99478
Name: count, dtype: int64

In [125]:
fi = model.get_feature_importance()
pd.Series(fi, index=X_full.columns).sort_values(ascending=False)

study_hours             30.840611
class_attendance        21.676511
study_sleeping_range    21.441226
sleep_quality_ord        9.958645
study_method             8.382000
facility_ord             6.522620
sleep_hours              0.693553
facility_rating          0.174119
sleep_quality            0.168156
age                      0.069636
course                   0.052141
gender                   0.007880
exam_difficulty          0.007166
internet_access          0.004211
exam_difficulty_ord      0.001523
dtype: float64

<bound method CatBoost.get_params of <catboost.core.CatBoostRegressor object at 0x000001A00096B040>>